In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Maranhão - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [32]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Maranhão - Produção de Cimento (t)', axis=1)
data

,Ano,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH,Maranhão - Consumo de Cimento (t)
0,2008,0.645315,7.978325,0.736322,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,0.592503,5.483335e+08,...,11.760983,25.354093,1004.987958,5445.077556,26.913732,23.175592,80.950690,1.958593,0.656244,963.205000
1,2009,0.650266,8.007891,0.737873,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,0.594639,5.302764e+08,...,9.472585,28.506238,1004.900392,3620.312713,26.992836,23.414460,81.602937,1.847599,0.658950,1123.586000
2,2010,0.654819,7.971707,0.739020,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,0.596141,5.046425e+08,...,9.332485,31.838993,1004.891838,2594.109885,26.968458,23.782363,83.292728,1.860782,0.661204,1378.741000
3,2011,0.658970,7.961524,0.739772,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,0.597021,4.854622e+08,...,10.985689,33.196582,1005.053412,2035.008505,27.022388,24.015667,84.183211,1.989042,0.663017,1282.016000
4,2012,0.662311,10.302202,0.740034,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,0.597229,4.867661e+08,...,8.159014,31.592321,1005.214749,1723.384057,26.989407,23.760558,83.318460,1.993051,0.664173,1528.770000
5,2013,0.666586,11.189661,0.740120,5.062157e+07,2.675613e+06,6.222354,4.060687e+07,0.596832,5.225848e+08,...,7.913561,29.227149,1005.449976,1548.437409,26.912712,23.364264,81.867671,1.963286,0.665521,1563.125000
6,2014,0.668885,11.986521,0.739796,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,0.595984,5.676413e+08,...,10.350442,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.665898,1303.411571
7,2015,0.671376,12.529442,0.739029,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,0.594816,5.179392e+08,...,12.454398,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.666139,1191.509657
8,2016,0.688469,12.762358,0.746544,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,0.606351,4.524144e+08,...,13.111908,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.677976,1100.217771
9,2017,0.689875,12.460135,0.745411,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,0.605565,4.213689e+08,...,9.479812,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.677792,971.921000


In [33]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,0.645315,7.978325,0.736322,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,0.592503,5.483335e+08,2.546644e+09,...,2.868108,11.760983,25.354093,1004.987958,5445.077556,26.913732,23.175592,80.950690,1.958593,0.656244
1,0.650266,8.007891,0.737873,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,0.594639,5.302764e+08,2.679945e+09,...,2.305885,9.472585,28.506238,1004.900392,3620.312713,26.992836,23.414460,81.602937,1.847599,0.658950
2,0.654819,7.971707,0.739020,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,0.596141,5.046425e+08,2.891930e+09,...,1.958246,9.332485,31.838993,1004.891838,2594.109885,26.968458,23.782363,83.292728,1.860782,0.661204
3,0.658970,7.961524,0.739772,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,0.597021,4.854622e+08,3.091988e+09,...,2.079088,10.985689,33.196582,1005.053412,2035.008505,27.022388,24.015667,84.183211,1.989042,0.663017
4,0.662311,10.302202,0.740034,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,0.597229,4.867661e+08,3.189506e+09,...,2.183282,8.159014,31.592321,1005.214749,1723.384057,26.989407,23.760558,83.318460,1.993051,0.664173
5,0.666586,11.189661,0.740120,5.062157e+07,2.675613e+06,6.222354,4.060687e+07,0.596832,5.225848e+08,3.093872e+09,...,2.403445,7.913561,29.227149,1005.449976,1548.437409,26.912712,23.364264,81.867671,1.963286,0.665521
6,0.668885,11.986521,0.739796,5.132137e+07,2.651392e+06,6.227127,4.069030e+07,0.595984,5.676413e+08,2.779340e+09,...,2.748397,10.350442,28.491261,1005.705374,1452.214268,27.001638,23.279428,81.037312,1.988340,0.665898
7,0.671376,12.529442,0.739029,5.174856e+07,2.602821e+06,6.218491,4.066452e+07,0.594816,5.179392e+08,2.561731e+09,...,2.501529,12.454398,29.089818,1005.751406,1363.412394,27.085524,23.327394,80.918789,1.953057,0.666139
8,0.688469,12.762358,0.746544,5.210281e+07,2.570101e+06,6.220890,4.069623e+07,0.606351,4.524144e+08,2.516859e+09,...,2.073464,13.111908,28.880429,1005.704586,1297.690355,27.029688,23.289212,81.028777,1.867218,0.677976
9,0.689875,12.460135,0.745411,5.221165e+07,2.559679e+06,6.221665,4.069725e+07,0.605565,4.213689e+08,2.497710e+09,...,2.177386,9.479812,27.965594,1005.796545,1279.407347,26.893536,23.259717,81.515345,1.794104,0.677792


In [34]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1123.586000
1     1378.741000
2     1282.016000
3     1528.770000
4     1563.125000
5     1303.411571
6     1191.509657
7     1100.217771
8      971.921000
9      750.012000
10     774.045000
11    1038.249000
12    1059.129000
13    1083.999000
14            NaN
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [35]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,-1.548004,-1.341931,-1.526781,-1.800669,0.888019,-1.813414,-1.863562,-1.353287,1.204752,-0.990234,...,2.026352,0.984445,-1.704426,-1.179358,2.780896,-0.581090,-1.106443,-1.032012,0.713734,-1.515780
1,-1.235625,-1.327021,-0.992611,-1.657451,0.872218,-1.896902,-1.851185,-0.893854,0.820657,-0.546688,...,0.085558,-0.051710,-0.236979,-1.419199,1.260440,0.474888,-0.140579,-0.401753,-0.347981,-1.152490
2,-0.948330,-1.345268,-0.597544,-1.115428,1.054815,-1.131157,-1.089628,-0.570965,0.275392,0.158672,...,-1.114490,-0.115145,1.314548,-1.442629,0.405374,0.149463,1.347032,1.231074,-0.221880,-0.849804
3,-0.686418,-1.350403,-0.338594,-0.599596,1.069928,-0.320349,-0.394850,-0.381737,-0.132596,0.824346,...,-0.697344,0.633403,1.946559,-1.000083,-0.060488,0.869373,2.290396,2.091538,1.004993,-0.606270
4,-0.475631,-0.170051,-0.248616,-0.122433,0.981661,0.385888,0.218857,-0.336916,-0.104860,1.148829,...,-0.337667,-0.646476,1.199713,-0.558185,-0.320144,0.429102,1.258865,1.255939,1.043345,-0.450985
5,-0.205832,0.277475,-0.218954,0.278382,0.616846,0.782503,0.619058,-0.422410,0.657046,0.830615,...,0.422337,-0.757614,0.098633,0.086092,-0.465916,-0.594704,-0.343548,-0.145943,0.758629,-0.269993
6,-0.060778,0.679314,-0.330312,0.591894,0.193718,0.915640,0.817536,-0.604630,1.615453,-0.215960,...,1.613108,0.345772,-0.243951,0.785621,-0.546092,0.592379,-0.686579,-0.948310,0.998286,-0.219397
7,0.096364,0.953097,-0.594440,0.783275,-0.654791,0.674736,0.756208,-0.855798,0.558230,-0.940033,...,0.760923,1.298413,0.034700,0.911699,-0.620085,1.712186,-0.492629,-1.062838,0.660786,-0.186967
8,1.174906,1.070551,1.993279,0.941980,-1.226398,0.741645,0.831633,1.624690,-0.835563,-1.089341,...,-0.716758,1.596124,-0.062778,0.783463,-0.674847,0.966821,-0.647020,-0.956557,-0.160319,1.402510
9,1.263639,0.918147,1.603303,0.990742,-1.408452,0.763263,0.834059,1.455606,-1.495939,-1.153056,...,-0.358019,-0.048438,-0.488670,1.035335,-0.690081,-0.850683,-0.766284,-0.486392,-0.859692,1.377796


In [36]:
reshaped_train_input = dfToInputRNN(train_input)

In [37]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 1.21119213,  0.75064964,  0.29363176,  0.73311646,
         -1.01788684,  0.18340237,  0.37196342,  1.03003753,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042, -1

In [38]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     1123.586000
1     1378.741000
2     1282.016000
3     1528.770000
4     1563.125000
5     1303.411571
6     1191.509657
7     1100.217771
8      971.921000
9      750.012000
10     774.045000
11    1038.249000
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [39]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1038.249
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [40]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
12,1.137321,0.665597,-0.713268,0.506899,-0.617267,0.098822,0.211599,0.829595,1.108405,2.635823,...,-1.482779,-2.260447,-0.619583,0.376717,-0.483958,-1.82319,-0.372439,0.371831,-1.374154,0.896376


In [41]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 1.13732127,  0.66559737, -0.71326822,  0.50689924,
         -0.61726733,  0.09882248,  0.21159948,  0.8295948 ,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738, -0

In [42]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1059.129
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [43]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [44]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [45]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1634392657, 1034148771, 378427692, 1307991656, 3846666093, 438390937, 3239114153, 3665072241, 1184710544, 1387684993, 2659254867, 1172153838, 2171843931, 3761652333, 1376890890, 3789361458, 978559474, 1212136950, 3131625690, 2554766708, 2631688986, 759004330, 957582700, 3821462486, 1658240370, 4055910624, 2166971830, 2917542287, 2335837066, 1082339119, 3450998069, 4207383262, 2263928309, 2075502288, 387628957, 3391155501, 3128144901, 2065728619, 3630566788, 2848775764, 2957136700, 2133018954, 1197821742, 3162769230, 2928188445, 3026214974, 1552733573, 276574919, 1001174260, 2821245590, 2749929657, 2866360629, 22223144, 354314011, 4269847337, 118588051, 474510535, 3855711560, 1499413034, 1690957775, 2272914859, 2530304006, 1233898076, 59732897, 1544594629, 1470005533, 2138259231, 1676127207, 3789288548, 498554762, 3803869471, 3219315278, 2164549657, 3131534426, 4078077943, 2334194922, 3642994579, 1374683500, 12055230, 749327148, 2209681373, 1102943860, 1208372519, 1265245026, 188324484

loss: 935.1447143554688


Step: 89 ___________________________________________
loss: 1578.8597412109375


Step: 90 ___________________________________________
loss: 1618.4498291015625


Step: 91 ___________________________________________
loss: 1845.3525390625


Step: 92 ___________________________________________
loss: 1436.9609375


Step: 93 ___________________________________________
loss: 1111.1270751953125


Step: 94 ___________________________________________
loss: 917.6316528320312


Step: 95 ___________________________________________
loss: 957.0962524414062


Step: 96 ___________________________________________
loss: 1550.54150390625


Step: 97 ___________________________________________
loss: 2172.078857421875


Step: 98 ___________________________________________
loss: 1615.5279541015625


Step: 99 ___________________________________________
loss: 2855.220947265625


final_seed: 759004330


In [46]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 1425061.0000
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 1393614.3750
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 1236921.5000
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 349903.8438
Epoch 5/10000
1/1 [==============================] - 0s 8ms/step - loss: 85415.7109
Epoch 6/10000
1/1 [==============================] - 0s 7ms/step - loss: 55539.1055
Epoch 7/10000
1/1 [==============================] - 0s 7ms/step - loss: 34890.8555
Epoch 8/10000
1/1 [==============================] - 0s 7ms/step - loss: 34197.3242
Epoch 9/10000
1/1 [==============================] - 0s 7ms/step - loss: 25966.9551
Epoch 10/10000
1/1 [==============================] - 0s 7ms/step - loss: 24187.5547
Epoch 11/10000
1/1 [==============================] - 0s 7ms/step - loss: 20623.2949
Epoch 12/10000
1/1 [==============================] - 0s 8ms/step - 

1/1 [==============================] - 0s 5ms/step - loss: 9745.1025
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 9934.4971
Epoch 99/10000
1/1 [==============================] - 0s 6ms/step - loss: 21957.4238
Epoch 100/10000
1/1 [==============================] - 0s 6ms/step - loss: 19449.4082
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 17312.6641
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 14162.6592
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 20749.1289
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 14941.2656
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 10729.3730
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 14167.4648
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 16961.3105
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss:

1/1 [==============================] - 0s 5ms/step - loss: 26574.7285
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 10178.0596
Epoch 194/10000
1/1 [==============================] - 0s 5ms/step - loss: 14376.1250
Epoch 195/10000
1/1 [==============================] - 0s 5ms/step - loss: 16409.6074
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 12599.0068
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 15283.7373
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 16012.7979
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 21648.9277
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 17044.6367
Epoch 201/10000
1/1 [==============================] - 0s 6ms/step - loss: 20878.2344
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 14411.7256
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 13073.2275
Epoch 288/10000
1/1 [==============================] - 0s 5ms/step - loss: 14206.3467
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 12670.1133
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 9304.2178
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 17886.1660
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 11484.6299
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 12915.9443
Epoch 294/10000
1/1 [==============================] - 0s 5ms/step - loss: 16429.8184
Epoch 295/10000
1/1 [==============================] - 0s 5ms/step - loss: 14563.9375
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 11479.6211
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 13199.4795
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 5ms/step - loss: 12234.1338
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 14265.9521
Epoch 384/10000
1/1 [==============================] - 0s 5ms/step - loss: 16990.8770
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 24157.7812
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 21872.6348
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 18446.9531
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 23305.3594
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 11927.4951
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 20624.8848
Epoch 391/10000
1/1 [==============================] - 0s 5ms/step - loss: 11399.2314
Epoch 392/10000
1/1 [==============================] - 0s 5ms/step - loss: 16686.9395
Epoch 393/10000
1/1 [==============================] - 0s 6ms/step - l

...

1/1 [==============================] - 0s 5ms/step - loss: 5802.5820
Epoch 1145/10000
1/1 [==============================] - 0s 4ms/step - loss: 9335.2041
Epoch 1146/10000
1/1 [==============================] - 0s 4ms/step - loss: 14800.3633
Epoch 1147/10000
1/1 [==============================] - 0s 5ms/step - loss: 11661.1982
Epoch 1148/10000
1/1 [==============================] - 0s 4ms/step - loss: 20347.6035
Epoch 1149/10000
1/1 [==============================] - 0s 4ms/step - loss: 3722.6370
Epoch 1150/10000
1/1 [==============================] - 0s 5ms/step - loss: 7175.2070
Epoch 1151/10000
1/1 [==============================] - 0s 5ms/step - loss: 12048.7842
Epoch 1152/10000
1/1 [==============================] - 0s 4ms/step - loss: 7572.6714
Epoch 1153/10000
1/1 [==============================] - 0s 5ms/step - loss: 17234.6230
Epoch 1154/10000
1/1 [==============================] - 0s 5ms/step - loss: 16933.2578
Epoch 1155/10000
1/1 [==============================] - 0s 4ms/st

1/1 [==============================] - 0s 5ms/step - loss: 11036.9971
Epoch 1239/10000
1/1 [==============================] - 0s 4ms/step - loss: 8980.0830
Epoch 1240/10000
1/1 [==============================] - 0s 4ms/step - loss: 11425.3750
Epoch 1241/10000
1/1 [==============================] - 0s 4ms/step - loss: 11069.2939
Epoch 1242/10000
1/1 [==============================] - 0s 4ms/step - loss: 14601.9092
Epoch 1243/10000
1/1 [==============================] - 0s 4ms/step - loss: 15491.7939
Epoch 1244/10000
1/1 [==============================] - 0s 4ms/step - loss: 13291.8174
Epoch 1245/10000
1/1 [==============================] - 0s 4ms/step - loss: 11402.0625
Epoch 1246/10000
1/1 [==============================] - 0s 5ms/step - loss: 6236.5142
Epoch 1247/10000
1/1 [==============================] - 0s 5ms/step - loss: 14578.8857
Epoch 1248/10000
1/1 [==============================] - 0s 5ms/step - loss: 13352.0205
Epoch 1249/10000
1/1 [==============================] - 0s 5ms

1/1 [==============================] - 0s 4ms/step - loss: 11925.5078
Epoch 1333/10000
1/1 [==============================] - 0s 4ms/step - loss: 8109.5210
Epoch 1334/10000
1/1 [==============================] - 0s 4ms/step - loss: 15960.7656
Epoch 1335/10000
1/1 [==============================] - 0s 4ms/step - loss: 10558.7500
Epoch 1336/10000
1/1 [==============================] - 0s 5ms/step - loss: 9066.3486
Epoch 1337/10000
1/1 [==============================] - 0s 5ms/step - loss: 10578.0889
Epoch 1338/10000
1/1 [==============================] - 0s 4ms/step - loss: 11620.2881
Epoch 1339/10000
1/1 [==============================] - 0s 4ms/step - loss: 12506.8604
Epoch 1340/10000
1/1 [==============================] - 0s 4ms/step - loss: 10963.8047
Epoch 1341/10000
1/1 [==============================] - 0s 4ms/step - loss: 10521.2354
Epoch 1342/10000
1/1 [==============================] - 0s 4ms/step - loss: 13230.4141
Epoch 1343/10000
1/1 [==============================] - 0s 5ms

1/1 [==============================] - 0s 4ms/step - loss: 18920.2207
Epoch 1427/10000
1/1 [==============================] - 0s 5ms/step - loss: 9514.5244
Epoch 1428/10000
1/1 [==============================] - 0s 4ms/step - loss: 14533.3984
Epoch 1429/10000
1/1 [==============================] - 0s 4ms/step - loss: 10157.4521
Epoch 1430/10000
1/1 [==============================] - 0s 5ms/step - loss: 9478.8027
Epoch 1431/10000
1/1 [==============================] - 0s 5ms/step - loss: 10511.8271
Epoch 1432/10000
1/1 [==============================] - 0s 4ms/step - loss: 7618.2793
Epoch 1433/10000
1/1 [==============================] - 0s 5ms/step - loss: 6167.6543
Epoch 1434/10000
1/1 [==============================] - 0s 4ms/step - loss: 9692.8945
Epoch 1435/10000
1/1 [==============================] - 0s 4ms/step - loss: 7406.6270
Epoch 1436/10000
1/1 [==============================] - 0s 4ms/step - loss: 10344.3828
Epoch 1437/10000
1/1 [==============================] - 0s 4ms/ste

1/1 [==============================] - 0s 5ms/step - loss: 8634.9658
Epoch 1521/10000
1/1 [==============================] - 0s 5ms/step - loss: 4120.1406
Epoch 1522/10000
1/1 [==============================] - 0s 4ms/step - loss: 13402.7549
Epoch 1523/10000
1/1 [==============================] - 0s 16ms/step - loss: 8112.9038
Epoch 1523: early stopping


In [47]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 568ms/step
[1059.129](test_target) - [[1029.621]](prediction) = 29.508028320312405


In [48]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.027860655614483608

In [49]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [50]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
0,-1.241459,-0.486246,-1.281082,-0.935735,-0.609310,-0.581902,-0.724135,-1.289469,1.148205,-1.122043,...,1.308949,1.412346,-1.213218,1.409635,1.321922,-1.348681,-1.129447,-1.011331,1.405833,-1.260158
1,0.034142,1.393198,0.121772,-0.450446,-0.800585,-0.825312,-0.689940,0.141790,0.140893,-0.184474,...,-0.190804,-0.643250,-0.022737,-0.606347,-0.225769,1.042844,-0.172339,-0.350435,-0.836052,0.074200
2,1.207317,-0.906952,1.159310,1.386181,1.409895,1.407214,1.414076,1.147679,-1.289098,1.306518,...,-1.118144,-0.769096,1.235955,-0.803288,-1.096153,0.305837,1.301786,1.361766,-0.569781,1.185958


train_target:


,Maranhão - Consumo de Cimento (t)
0,1123.586
1,1378.741
2,1282.016


test_input:


...

,Maranhão - IDH Educacao,Maranhão - Desemprego,Maranhão - IDH Longevidade,Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Maranhão - IDH
12,1.137321,0.665597,-0.713268,0.506899,-0.617267,0.098822,0.211599,0.829595,1.108405,2.635823,...,-1.482779,-2.260447,-0.619583,0.376717,-0.483958,-1.82319,-0.372439,0.371831,-1.374154,0.896376


test_target:


,Maranhão - Consumo de Cimento (t)
12,1059.129


1/1 [==============================] - 1s 550ms/step
Error: 29.508028320312405




[1259.790771484375,
 1457.5809326171875,
 1057.6727294921875,
 1195.0020751953125,
 1198.633056640625,
 1109.36767578125,
 994.6742553710938,
 809.2565307617188,
 743.8080444335938,
 1029.6209716796875]

In [51]:
display(targets)
display(predictions)

[1528.77,
 1563.125,
 1303.4115714285715,
 1191.5096571428571,
 1100.2177714285715,
 971.921,
 750.012,
 774.045,
 1038.249,
 1059.129]

[1259.790771484375,
 1457.5809326171875,
 1057.6727294921875,
 1195.0020751953125,
 1198.633056640625,
 1109.36767578125,
 994.6742553710938,
 809.2565307617188,
 743.8080444335938,
 1029.6209716796875]

In [52]:
mae = mean_absolute_error(predictions, targets)
mae

146.34392869001115

In [53]:
porcentage = mae/np.mean(targets)
porcentage

0.12973303998355656